In [3]:
!pip install assemblyai
!pip install openai==1.2.0
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
import openai
from openai import OpenAI
client = OpenAI(
  api_key="Enter_Your_OpenAI_Key_Here"
)
def get_completion(prompt, model = "gpt-4-turbo-preview", temperature =0):
  messages = [{"role":"user", "content": prompt}]
  response = client.chat.completions.create(
      model = model,
      messages = messages,
      temperature =temperature # degree of expiration (randomness) (0 same result - 1 creative)
  )
  return response.choices[0].message.content

In [5]:
import os
import assemblyai as aai
import pandas as pd
import json


In [48]:
wav_dir = 'Directory for the recording .wav'
trans_dir = 'Directory to store transcript files'
aai.settings.api_key = "Enter_Your_AssemblyAI_Key_Here"
names = os.listdir(wav_dir)
config = aai.TranscriptionConfig(
  speaker_labels=True,
  speakers_expected=2
)


## Diarization


In [54]:
def get_words(transcript):
  speaker = []
  words = []
  start_time = []
  end_time = []
  for utterance in transcript.utterances:
    for word in utterance.words:
      speaker.append(word.speaker)
      words.append(word.text)
      start_time.append(word.start)
      end_time.append(word.end)
  result = pd.DataFrame({'Speaker': speaker, 'Utterance start time (milliseconds)': start_time, 'Utterance end time (milliseconds)': end_time,  'Utterance': words})
  return result

In [55]:
def combine_sentence(result):
  speaker = []
  text = []
  start_time = []
  end_time = []
  temp = ''
  s_time = result['Utterance start time (milliseconds)'][0]
  for i in range(len(result)):
    seg = result['Utterance'][i]
    temp = temp + " " + seg
    if (seg[-1] in ['?','.']):
      speaker.append(result['Speaker'][i])
      text.append(temp)
      start_time.append(s_time)
      end_time.append(result['Utterance end time (milliseconds)'][i])
      if (i+1 < len(result)):
        s_time = result['Utterance start time (milliseconds)'][i+1]
      temp  = ''
  result_1 = pd.DataFrame({'Speaker': speaker, 'Utterance start time (milliseconds)': start_time, 'Utterance end time (milliseconds)': end_time,  'Utterance': text})
  return result_1

In [ ]:
for n in names:
  path = wav_dir + n
  transcript = aai.Transcriber().transcribe(path, config)
  if transcript.utterances is None:
    result = pd.DataFrame({'Speaker': [], 'Utterance start time (milliseconds)': [], 'Utterance end time (milliseconds)': [],  'Utterance': []})
  else:
    result = get_words(transcript)
  if len(result) != 0:
    result_1 = combine_sentence(result)

    temp = [result['Speaker'][i]+ " : " + result['Utterance'][i]  for i in range(len(result))]
    d = ' '.join(temp) [:900]
    prompt = f"""I will provide you a transcript of a tutor and a student.
    There will to two speakers A and B. You task is to identify which is student and which is tutor.
    In this session, a tutor may teach students knowledge about the topic, help student solve their problem, or ask questions to check the students' understanding.
    Format your output in a JSON format with A,B as key; student, tutor as value
    """ + d
    r = get_completion(prompt)
    a = json.loads(r[8:-4])
    result_1 = result_1.replace({'Speaker': a})
    result_1.to_csv(trans_dir+n[:-4]+'.csv', index=False)
  else:
    result.to_csv(trans_dir+n[:-4]+'.csv', index=False)
